In [1]:
# !pip install langchain milvus python-dotenv pymilvus

In [10]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Milvus

超参数

In [3]:
MILVUS_HOST = "localhost"
MILVUS_PORT = "19530"

数据加载器

In [4]:
# pip install bs4
loader = WebBaseLoader([
    "https://milvus.io/docs/overview.md",
])
docs = loader.load()
# 数据分割
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
docs = text_splitter.split_documents(docs)


Created a chunk of size 1743, which is longer than the specified 1024
Created a chunk of size 1278, which is longer than the specified 1024


In [5]:
docs[0].page_content

'Introduction Milvus documentationDocsTutorialsToolsBlogCommunityStars0Join SlackTry Managed Milvus FREESearchHomev2.2.x\u200bAbout MilvusWhat is MilvusMilvus AdoptersMilvus RoadmapMilvus LimitsReleasesEnhancement ProposalsBootcampGet StartedUser GuideAdministration GuideIntegrationsBenchmarksToolsReferenceExample ApplicationsFAQsAPI referenceIntroduction\nThis page aims to give you an overview of Milvus by answering several questions. After reading this page, you will learn what Milvus is and how it works, as well as the key concepts, why use Milvus, supported indexes and metrics, example applications, the architecture, and relevant tools.\nWhat is Milvus vector database?\nMilvus was created in 2019 with a singular goal: store, index, and manage massive embedding vectors generated by deep neural networks and other machine learning (ML) models.\nAs a database specifically designed to handle queries over input vectors, it is capable of indexing vectors on a trillion scale. Unlike existi

指定embedding模型，非 OpenAIEmbeddings()。
### 可以指定本地模型哦！


1. 普通Bert类embedding

```python
from langchain.embeddings import HuggingFaceEmbeddings

# 为空，默认是 "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings()
query_result = embeddings.embed_query(query)
```

2. 指令类embedding

```python
from langchain.embeddings import HuggingFaceInstructEmbeddings
# 默认是："hkunlp/instructor-large"
embeddings = HuggingFaceInstructEmbeddings(
    model_name = "hkunlp/instructor-xl",
    embed_instruction = "Represent the document for retrieval: "
)
```

In [6]:
# 指令型 embedding，中文的可以选择其他的模型
from langchain.embeddings import HuggingFaceInstructEmbeddings
# 默认是："hkunlp/instructor-large"
embeddings = HuggingFaceInstructEmbeddings(
    model_name = "hkunlp/instructor-xl",
    embed_instruction = "Represent the document for retrieval: ",
)

c:\Users\admin\.conda\envs\chens\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load INSTRUCTOR_Transformer
max_seq_length  512


将上面的docs文档，转换成embedding 向量， Milvus存储

In [7]:
milvus_db = Milvus.from_documents(
    docs,
    embedding=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT}
)

In [8]:
query = "What is milvus?"
smi_docs = milvus_db.similarity_search(query)

In [9]:
from pprint import pprint
pprint(smi_docs[0])

Document(page_content='Milvus workflow.', metadata={'source': 'https://milvus.io/docs/overview.md', 'title': 'Introduction Milvus documentation', 'description': 'Milvus is an open-source vector database designed specifically for AI application development, embeddings similarity search, and MLOps v2.2.x.', 'language': 'en'})
